In [2]:
from sqlalchemy import create_engine, Table, Column, Integer, String, MetaData, Date, ForeignKey, insert, select
from sqlalchemy.orm import declarative_base, relationship, Session, sessionmaker
from sqlalchemy_utils import create_database, database_exists
import json
import random
import numpy as np
import pandas as pd
from datetime import date

In [3]:
Base = declarative_base()

db_master = create_engine("mysql+pymysql://root:@localhost/information_schema")
db_master.execute("DROP DATABASE IF EXISTS mppl")

In [4]:
engine = create_engine("mysql+pymysql://root:@localhost/mppl")
if not database_exists(engine.url):
    create_database(engine.url)

In [5]:
meta = MetaData()

In [6]:
aparatwarga = Table(
    "Aparat_warga_assoc",
    Base.metadata,
    Column("aparat_id", ForeignKey("AparatDesa.AparatID")),
    Column("warga_id", ForeignKey("WargaDesa.WargaID")),
)

In [7]:
class dusun(Base):
    __tablename__ = "Dusun"
    DusunID = Column(Integer, autoincrement=True, primary_key=True)
    Nama_Dusun = Column(String(64))
    dusunku = relationship("wargadesa")

In [8]:
class wargadesa(Base):
    __tablename__ = "WargaDesa"
    WargaID = Column(Integer, autoincrement=True, primary_key=True)
    NIK = Column(String(16))
    Nama = Column(String(64))
    Nomor_KK = Column(String(16))
    Jenis_Kelamin = Column(String(32))
    Status_Perkawinan = Column(String(32))
    Tanggal_Lahir = Column(Date)
    Pekerjaan = Column(String(64))
    Status_Dalam_Keluarga = Column(String(64))
    Nomor_Telepon = Column(String(16))
    aparat = relationship("aparat_desa", secondary=aparatwarga)
    dusun_id = Column(Integer, ForeignKey("Dusun.DusunID"))

In [9]:
class aparat_desa(Base):
    __tablename__ = "AparatDesa"
    AparatID = Column(Integer, autoincrement=True, primary_key=True)
    Nama = Column(String(64))
    Email = Column(String(128))

In [10]:
Base.metadata.create_all(engine)

In [11]:
with open("data_json.json", "r") as openfile:
    data_masukan = json.load(openfile)

In [12]:
for nama_dusun in data_masukan["nama_dusun"]:
    stmt = insert(dusun).values(Nama_Dusun=nama_dusun)
    with engine.connect() as conn:
        conn.execute(stmt)

In [13]:
connection = engine.connect()
stmt = select(dusun)
data = connection.execute(stmt)
data = list(data)
kolom_dusun = dusun.__table__.columns.keys()
hasil = list(map(list, zip(*data)))
indeks = 0
for i in kolom_dusun:
    globals()[i] = hasil[indeks]
    indeks += 1
DusunID

[1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11]

## Masukkan Data Warga Desa

In [14]:
data_masukan.keys()

dict_keys(['nama_laki_laki', 'nama_perempuan', 'nama_dusun', 'status_perkawinan', 'status_dalam_keluarga', 'jenis_kelamin', 'tanggal_lahir', 'pekerjaan'])

In [15]:
data_masukan["status_perkawinan"]

['Belum Kawin', 'Kawin', 'Cerai Hidup', 'Cerai Mati']

In [16]:
data_masukan["tanggal_lahir"] = np.array(data_masukan["tanggal_lahir"],dtype='datetime64')
data_masukan["tanggal_lahir"]

array(['1945-01-01', '1945-01-02', '1945-01-03', ..., '2022-12-29',
       '2022-12-30', '2022-12-31'], dtype='datetime64[D]')

In [17]:
from datetime import date
tanggal_belum_kawin = data_masukan["tanggal_lahir"][((date.today().year - data_masukan["tanggal_lahir"].astype('datetime64[Y]').astype("int64") - 1970) >= 17) & ((date.today().year - data_masukan["tanggal_lahir"].astype('datetime64[Y]').astype("int64") - 1970) < 25)]
tanggal_kawin = data_masukan["tanggal_lahir"][((date.today().year - data_masukan["tanggal_lahir"].astype('datetime64[Y]').astype("int64") - 1970) >= 25)]

In [18]:
set(data_masukan["status_perkawinan"])

{'Belum Kawin', 'Cerai Hidup', 'Cerai Mati', 'Kawin'}

In [20]:
set(data_masukan["jenis_kelamin"])

{'Laki-Laki', 'Perempuan'}

In [19]:
data_masukan.keys()

dict_keys(['nama_laki_laki', 'nama_perempuan', 'nama_dusun', 'status_perkawinan', 'status_dalam_keluarga', 'jenis_kelamin', 'tanggal_lahir', 'pekerjaan'])

In [50]:
# class wargadesa(Base):
#     __tablename__ = "WargaDesa"
#     WargaID = Column(Integer, autoincrement=True, primary_key=True)
#     NIK = Column(String(16))
#     Nama = Column(String(64))
#     Nomor_KK = Column(String(16))
#     Jenis_Kelamin = Column(String(32))
#     Status_Perkawinan = Column(String(32))
#     Tanggal_Lahir = Column(Date)
#     Pekerjaan = Column(String(64))
#     Status_Dalam_Keluarga = Column(String(64))
#     Nomor_Telepon = Column(String(16))

# buat seberapa keluarganya
kk_rand_4dig = [str(i).zfill(4) for i in range(10000)]

import datetime
for _ in range(10000):
    Nomor_KK = "331223"
    status_kawin = random.choice(data_masukan["status_perkawinan"])
    if(status_kawin == "Belum Kawin"):
        tanggal_lahir = random.choice(tanggal_belum_kawin)
        usia = date.today().year - (1970 + tanggal_lahir.astype('datetime64[Y]').astype("int64"))
        tanggal_kk = tanggal_lahir - np.timedelta64(17*365,'D')
        tanggal_kk = tanggal_kk.astype(datetime.datetime)
        tanggal_kk  = tanggal_kk.strftime('%d%m%y')
        Nomor_KK += tanggal_kk
        Nomor_KK += random.choice(kk_rand_4dig)
        jenis_kelamin = random.choice(data_masukan["jenis_kelamin"])
        nama = ''
        if(jenis_kelamin=='Laki-Laki'):
            first_name = random.choice(data_masukan["nama_laki_laki"])
            last_name  = random.choice(data_masukan["nama_laki_laki"])
            nama = first_name + last_name
        else:
            first_name = random.choice(data_masukan["nama_perempuan"])
            last_name  = random.choice(data_masukan["nama_perempuan"])
        pekerjaan = random.choice(data_masukan['pekerjaan'])
        

    elif(status_kawin == "Kawin"):
        tanggal_lahir = random.choice(tanggal_kawin)
        usia = date.today().year - (1970 + tanggal_lahir.astype('datetime64[Y]').astype("int64"))
        tanggal_kk = tanggal_lahir - np.timedelta64(25*365,'D')
        tanggal_kk = tanggal_kk.astype(datetime.datetime)
        tanggal_kk  = tanggal_kk.strftime('%d%m%y')
        Nomor_KK += tanggal_kk
        Nomor_KK += random.choice(kk_rand_4dig)
        
        # print(np.datetime_as_string(tanggal_lahir,unit='Y'))
        # print(usia,tanggal_kk,tanggal_lahir)
    # elif(status_kawin == "Kawin"):

    #     if(tanggal_lahir)
    # elif(status_kawin == "Kawin"):
    #     b

16
16
16
16
16
16
16
16
16
16
16
16
16
16
16
16
16
16
16
16
16
16
16
16
16
16
16
16
16
16
16
16
16
16
16
16
16
16
16
16
16
16
16
16
16
16
16
16
16
16
16
16
16
16
16
16
16
16
16
16
16
16
16
16
16
16
16
16
16
16
16
16
16
16
16
16
16
16
16
16
16
16
16
16
16
16
16
16
16
16
16
16
16
16
16
16
16
16
16
16
16
16
16
16
16
16
16
16
16
16
16
16
16
16
16
16
16
16
16
16
16
16
16
16
16
16
16
16
16
16
16
16
16
16
16
16
16
16
16
16
16
16
16
16
16
16
16
16
16
16
16
16
16
16
16
16
16
16
16
16
16
16
16
16
16
16
16
16
16
16
16
16
16
16
16
16
16
16
16
16
16
16
16
16
16
16
16
16
16
16
16
16
16
16
16
16
16
16
16
16
16
16
16
16
16
16
16
16
16
16
16
16
16
16
16
16
16
16
16
16
16
16
16
16
16
16
16
16
16
16
16
16
16
16
16
16
16
16
16
16
16
16
16
16
16
16
16
16
16
16
16
16
16
16
16
16
16
16
16
16
16
16
16
16
16
16
16
16
16
16
16
16
16
16
16
16
16
16
16
16
16
16
16
16
16
16
16
16
16
16
16
16
16
16
16
16
16
16
16
16
16
16
16
16
16
16
16
16
16
16
16
16
16
16
16
16
16
16
16
16
16
16
16
16
16
16
16
16
16
16
16
16
16
1

In [21]:
# from dateutil.relativedelta import relativedelta
# for _ in range(5):
#     Nomor_KK = "331223"
#     status_kawin = random.choice(data_masukan["status_perkawinan"])
#     if(status_kawin == "Belum Kawin"):
#         tanggal_lahir = random.choice(tanggal_tahun_17)
#         print(tanggal_lahir)
#         # print(type(tanggal_lahir))
#         umurnya = date.today().year - pd.to_datetime(tanggal_lahir).year
#         if(umurnya > 20):
#             print(tanggal_lahir.replace(year=tanggal_lahir.year-10))

In [ ]:
print(type(relativedelta(years=3)))

<class 'dateutil.relativedelta.relativedelta'>


In [ ]:
from datetime import datetime,timedelta